<a href="https://colab.research.google.com/github/miguellucero123/Analisis_est/blob/main/Trabajo_AAAD_S2_ML_Beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importación de bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid') # Cambiado de 'seaborn'
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Definir la ruta al archivo
ruta = '/content/drive/MyDrive/26_05_2025/datos_supermercadousa.csv'

In [ ]:
# Parte 1
import pandas as pd

try:
    df_raw = pd.read_csv(ruta, encoding='latin-1', sep=';')
   # Mostrar la información y las primeras filas para verificar
    print("Columnas después de leer el CSV:")
    print(df_raw.columns) # Imprime los nombres de las columnas leídas
    display(df_raw.info())
    display(df_raw.head())

except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta '{ruta}'.")
except pd.errors.ParserError:
    print("Error: No se pudo analizar el archivo CSV. Verifique el formato, la codificación y los separadores.")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

In [ ]:
# Función para intentar convertir un valor a float y capturar errores
def try_float(value):
    try:
        # Intentar convertir, reemplazando la coma si existe y usando el punto como decimal
        # También manejar posibles espacios en blanco u otros caracteres
        return float(str(value).strip().replace(',', ''))
    except ValueError:
        return f"ERROR: '{value}'"
    except Exception as e:
        return f"ERROR inesperado: {e} para valor '{value}'"

# Aplicar la función a la columna 'Total' y buscar resultados que empiecen con 'ERROR'
problemas_total = df_raw['Total'].apply(try_float)
# Corrige 'problems_total' a 'problemas_total'
valores_con_error = problemas_total[problemas_total.astype(str).str.startswith('ERROR')]

if not valores_con_error.empty:
    print("\nValores en la columna 'Total' que causan problemas en la conversión:")
    display(valores_con_error.head()) # Mostrar los primeros valores problemáticos
else:
    print("\nNo se encontraron valores que causen errores de conversión obvios en 'Total'.")

In [ ]:
# Función corregida para intentar convertir un valor a float, manejando puntos como separadores de miles
def try_float_corrected(value):
    try:
        s_value = str(value).strip().replace(',', '') # Eliminar comas si existen

        # Manejar puntos: si hay más de uno, eliminar todos menos el último
        if s_value.count('.') > 1:
            parts = s_value.split('.')
            s_value_cleaned = ''.join(parts[:-1]) + '.' + parts[-1]
        else:
            s_value_cleaned = s_value # Si hay 0 o 1 punto, no necesita limpieza especial de puntos

        return float(s_value_cleaned)
    except ValueError:
        return f"ERROR: '{value}' (No se pudo convertir a número después de limpiar)"
    except Exception as e:
        return f"ERROR inesperado: {e} para valor '{value}'"

# Aplicar la función corregida a la columna 'Total' y buscar resultados que empiecen con 'ERROR'
problemas_total_corregido = df_raw['Total'].apply(try_float_corrected)
valores_con_error_corregido = problemas_total_corregido[problemas_total_corregido.astype(str).str.startswith('ERROR')]

if not valores_con_error_corregido.empty:
    print("\nValores en la columna 'Total' que AÚN causan problemas en la conversión (con la función corregida):")
    display(valores_con_error_corregido.head()) # Mostrar los primeros valores problemáticos restantes
else:
    print("\nNo se encontraron valores que causen errores de conversión obvios en 'Total' con la función corregida.")

In [ ]:
def try_float_corrected(value):
    try:
        s_value = str(value).strip().replace(',', '') # Eliminar comas si existen

        # Manejar puntos: si hay más de uno, eliminar todos menos el último
        if s_value.count('.') > 1:
            parts = s_value.split('.')
            # Eliminar todos los puntos excepto el último
            s_value_cleaned = ''.join(parts[:-1]) + '.' + parts[-1]
        else:
            # Si hay 0 o 1 punto, no necesita limpieza especial de puntos
            s_value_cleaned = s_value

        return float(s_value_cleaned)
    except ValueError:
        return f"ERROR: '{value}' (No se pudo convertir a número después de limpiar)"
    except Exception as e:
        return f"ERROR inesperado: {e} para valor '{value}'"

# Probar con un valor problemático del error
valor_ejemplo = '5.489.715'
resultado = try_float_corrected(valor_ejemplo)
print(f"Resultado para '{valor_ejemplo}': {resultado}")

# Probar con el valor real esperado para ver cómo se comporta la función si lo recibe
valor_real_esperado = '548.9715'
resultado_real_esperado = try_float_corrected(valor_real_esperado)
print(f"Resultado para '{valor_real_esperado}': {resultado_real_esperado}")

In [ ]:
# Parte 2
import pandas as pd

try:
    df_raw = pd.read_csv(ruta, encoding='latin-1', sep=';')
    display(df_raw.info())
    display(df_raw.head())
except FileNotFoundError:
    print("Error: No se encontró '{ruta}'.")
except pd.errors.ParserError:
    print("Error: No se pudo analizar el archivo CSV. Verifique el formato y la codificación.")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

In [ ]:
# Parte 3: Manejar valores faltantes, duplicados, atípicos y calcular columnas nuevas
import pandas as pd # Asegurar importación si esta celda se ejecuta sola

# Assuming df_raw exists from previous steps
if 'df_raw' not in locals():
    print("Error: df_raw no encontrado. Ejecute las partes anteriores.")
else:
    df_sin_faltantes = df_raw.dropna()
    df_sin_duplicados = df_sin_faltantes.drop_duplicates()

    # Función corregida (eliminado regex=False)
    def clean_number_string(value):
        s_value = str(value).strip().replace(',', '') # Eliminar comas (sin regex=False)

        # Manejar puntos: si hay más de uno, eliminar todos menos el último
        if s_value.count('.') > 1:
            parts = s_value.split('.')
            return ''.join(parts[:-1]) + '.' + parts[-1]
        else:
            return s_value # Si hay 0 o 1 punto, no necesita limpieza especial de puntos

    # Asegúrate de que Precio por unidad y Canrtidad son numéricas
    # Aplica la limpieza si es necesario
    # Usar errors='coerce' convierte errores a NaN, lo cual es manejado por dropna() o el filtrado.
    df_sin_duplicados['Precio por unidad'] = pd.to_numeric(df_sin_duplicados['Precio por unidad'].apply(clean_number_string), errors='coerce')
    # Asumiendo que Cantidad no tiene decimales ni separadores, pero aplicar pd.to_numeric por seguridad
    df_sin_duplicados['Canrtidad'] = pd.to_numeric(df_sin_duplicados['Canrtidad'], errors='coerce')
    # También limpia y convierte Ingreso bruto original si quieres compararlo o tener una versión numérica
    df_sin_duplicados['Ingreso bruto_Original_Numeric'] = pd.to_numeric(df_sin_duplicados['Ingreso bruto'].apply(clean_number_string), errors='coerce')


    # Calcular el subtotal antes de impuestos
    df_sin_duplicados['Subtotal_Calculado'] = df_sin_duplicados['Precio por unidad'] * df_sin_duplicados['Canrtidad']

    # Calcular el Impuesto esperado (5% del subtotal)
    df_sin_duplicados['Impuesto_Calculado'] = df_sin_duplicados['Subtotal_Calculado'] * 0.05

    # Calcular el Total esperado (Subtotal + Impuesto Calculado)
    df_sin_duplicados['Total_Calculado'] = df_sin_duplicados['Subtotal_Calculado'] + df_sin_duplicados['Impuesto_Calculado']

    # Calcular Ingreso Bruto basado en tu aclaración: es igual al Impuesto Calculado
    df_sin_duplicados['Ingreso bruto_Calculado'] = df_sin_duplicados['Impuesto_Calculado']

    # === ADDED DEBUGGING STEPS ===
    print("\nInfo de df_sin_duplicados después de calcular columnas:")
    display(df_sin_duplicados.info())
    print("\nColumnas de df_sin_duplicados después de calcular columnas:")
    print(df_sin_duplicados.columns)
    display(df_sin_duplicados[['Precio por unidad', 'Canrtidad', 'Subtotal_Calculado', 'Impuesto_Calculado', 'Total_Calculado', 'Ingreso bruto_Calculado']].head())
    # === END DEBUGGING STEPS ===


    # Calcular Margen Bruto esperado (Gross Margin percentage) - Si lo necesitas y tienes COGS
    # Si el Ingreso Bruto ahora es el Impuesto, la definición de Margen Bruto cambia drásticamente.
    # Podrías no necesitar Margen Bruto calculado con esta definición de Ingreso Bruto.
    # Si lo necesitas, tendrás que aclarar qué representa COGS en este contexto.


    # Ahora, utiliza 'Total_Calculado' para el filtrado de atípicos
    # Asegúrate de que 'Total_Calculado' y 'Rating' no tengan NaNs que impidan el cálculo del cuantil.
    # Añadir un fillna(0) o dropna() temporalmente si sospechas de NaNs en estas columnas antes de quantile
    # Por ejemplo:
    # df_temp_for_quantile = df_sin_duplicados.dropna(subset=['Total_Calculado', 'Rating'])
    # Q1_total_calc = df_temp_for_quantile['Total_Calculado'].quantile(0.25)
    # Q3_total_calc = df_temp_for_quantile['Total_Calculado'].quantile(0.75)

    # Asumimos que dropna() inicial manejó la mayoría de NaNs, pero si pd.to_numeric creó más, quantile podría fallar.
    # El error original no fue en quantile, así que asumimos que estos existen para el cálculo del cuantil.
    Q1_total_calc = df_sin_duplicados['Total_Calculado'].quantile(0.25)
    Q3_total_calc = df_sin_duplicados['Total_Calculado'].quantile(0.75)
    IQR_total_calc = Q3_total_calc - Q1_total_calc
    limite_inferior_total = Q1_total_calc - 1.5 * IQR_total_calc
    limite_superior_total = Q3_total_calc + 1.5 * IQR_total_calc

    # Para Rating, asumo que ya es numérico
    Q1_rating = df_sin_duplicados['Rating'].quantile(0.25)
    Q3_rating = df_sin_duplicados['Rating'].quantile(0.75)
    IQR_rating = Q3_rating - Q1_rating
    limite_inferior_rating = Q1_rating - 1.5 * IQR_rating
    limite_superior_rating = Q3_rating + 1.5 * IQR_rating

    # === Ensure 'Total_Calculado' and 'Rating' are not all NaNs before filtering ===
    if df_sin_duplicados['Total_Calculado'].isnull().all():
         print("Warning: 'Total_Calculado' is all NaNs before filtering.")
    if df_sin_duplicados['Rating'].isnull().all():
         print("Warning: 'Rating' is all NaNs before filtering.")
    # === END NaN check ===


    df_limpio = df_sin_duplicados[
        (df_sin_duplicados['Total_Calculado'].between(limite_inferior_total, limite_superior_total, inclusive='both')) &
        (df_sin_duplicados['Rating'].between(limite_inferior_rating, limite_superior_rating, inclusive='both'))
    ].copy() # Usar .copy() para evitar SettingWithCopyWarning

    # Inspeccionar df_limpio después de calcular columnas nuevas y filtrar
    print("\nColumnas de df_limpio después de calcular columnas nuevas y filtrar:")
    print(df_limpio.columns)
    display(df_limpio[['Precio por unidad', 'Canrtidad', 'Subtotal_Calculado', 'Impuesto_Calculado', 'Total_Calculado', 'Ingreso bruto_Calculado', 'Total', 'Impuesto 5%', 'Ingreso bruto']].head()) # Compara
    display(df_limpio['Total_Calculado'].info())
    display(df_limpio['Impuesto_Calculado'].info())
    display(df_limpio['Ingreso bruto_Calculado'].info())

    # Guardar los datos limpios (opcional, asegúrate de que solo guardas las columnas que necesitas)
    # df_limpio.to_csv('df_limpio_calculado.csv', index=False)
    display(df_limpio.head())
    # Guardar los datos limpios
    df_limpio.to_csv('df_limpio.csv', index=False)
    display(df_limpio.head())

In [ ]:
# Parte 4: Análisis exploratorio de datos (Usando df_limpio con columnas calculadas y renombradas)

# Examinar las primeras filas (deberían reflejar los cambios de Parte 3 y 5)
print("\nPrimeras filas de df_limpio después de limpieza, cálculo y renombrado:")
display(df_limpio.head())

# Verificar tipos de datos y valores faltantes
print("\nInformación de df_limpio después de limpieza, cálculo y renombrado:")
display(df_limpio.info()) # Verifica que las columnas de interés son numéricas (float64)

# Estadísticas resumidas para columnas numéricas (usando los nombres finales)
print("\nEstadísticas descriptivas de df_limpio:")
display(df_limpio.describe()) # Verifica las estadísticas para Total, Tax 5%, Gross income, Quantity, Rating

# Valores únicos en columnas categóricas (usando los nombres finales)
print("\nConteo de valores para columnas categóricas:")
for col in ['Gender', 'Product line', 'City', 'Payment', 'Customer type']:
    if col in df_limpio.columns:
        display(df_limpio[col].value_counts())
    else:
        print(f"Columna categórica '{col}' no encontrada en el DataFrame (Verifica Parte 5).")

# Investigar relaciones entre columnas (usando los nombres finales)
print("\nAnálisis de relaciones entre columnas:")
# ASEGÚRATE DE USAR LOS NOMBRES DE COLUMNA FINALES DESPUÉS DE LA PARTE 5
# La columna renombrada de 'Linea de producto' a 'Product line' debe usarse aquí.
# La columna 'Total' (que fue renombrada de 'Total_Calculado') debe usarse aquí.

# Verificar dtype de la columna 'Total' antes de agrupar
if df_limpio['Total'].dtype == 'object':
    print("\nWARNING: 'Total' column is still of object dtype. Attempting to convert to numeric.")
    # Apply the cleaning function and convert to numeric again, coercing errors
    df_limpio['Total'] = pd.to_numeric(df_limpio['Total'].apply(clean_number_string), errors='coerce')
    # If there are still NaNs created by 'coerce', you might need to handle them (e.g., drop or fill)
    # df_limpio.dropna(subset=['Total'], inplace=True) # Uncomment if needed
    print("Conversion attempt complete. Checking dtype again:")
    display(df_limpio['Total'].info()) # Check info after conversion attempt

# Ahora usa 'Product line' y 'Total' para el agrupamiento y agregación
if 'Product line' in df_limpio.columns and 'Total' in df_limpio.columns:
    display(df_limpio.groupby('Product line')['Total'].agg(['mean', 'sum'])) # CORRECTED COLUMN NAME
    display(df_limpio.groupby(['Product line', 'Gender'])['Rating'].mean())
    display(df_limpio.groupby(['City', 'Gender', 'Product line'])['Quantity'].sum())
else:
    print("\nError: 'Product line' or 'Total' columns not found after renaming. Check Parte 5.")

# Puedes añadir más análisis exploratorios aquí


# Resumen de observaciones (actualizado con los nombres y lógica de las columnas)
print("\nResumen de observaciones:")
print("Cada registro en df_limpio representa una única transacción en el supermercado.")
print("Las columnas clave, después de limpieza y cálculo, incluyen:")
print("- Invoice ID: Identificador único para cada transacción.")
print("- Branch: La sucursal donde se realizó la transacción.")
print("- City: La ciudad donde se encuentra la sucursal.")
print("- Customer type: Si el cliente es miembro o cliente normal.")
print("- Gender: El género del cliente.")
print("- Product line: La categoría del producto comprado.")
print("- Unit price: El precio de una unidad del producto (limpiado y convertido).")
print("- Quantity: El número de unidades compradas (Canrtidad original, renombrada).")
print("- Subtotal: El subtotal calculado (Precio por unidad * Quantity).")
print("- Tax 5%: El monto de impuesto calculado (5% del Subtotal).")
print("- Total: El monto total calculado (Subtotal + Tax 5%).")
print("- Date: La fecha de la transacción.")
print("- Time: La hora de la transacción.")
print("- Payment: El método de pago utilizado.")
print("- COGS: Costo de los bienes vendidos (si la columna original se mantuvo o calculó).") # Ajusta si eliminaste/recalculaste cogs
print("- Gross margin percentage: El porcentaje de margen bruto (si la columna original se mantuvo o calculó).") # Ajusta si eliminaste/recalculaste
print("- Gross income: El ingreso bruto calculado (igual al Tax 5% calculado, según tu aclaración).")
print("- Rating: Calificación del cliente para la transacción.")

# Opcional: Guardar los datos limpios (con los nombres finales)
# df_limpio.to_csv('df_limpio_final.csv', index=False)
# print("\nDataFrame limpio guardado como 'df_limpio_final.csv'")

In [ ]:
# Parte 5: Renombrar y eliminar columnas
# Ahora que tenemos las columnas calculadas, renombramos y eliminamos las originales si no son necesarias.

# Renombrar columnas (incluyendo la corrección de 'Canrtidad' y otras que no son las calculadas clave)
# Asegúrate de que las claves del diccionario coincidan EXACTAMENTE
# con los nombres de las columnas originales que quieres renombrar.
df_limpio = df_limpio.rename(columns={
    'Canrtidad': 'Quantity', # Corregir error tipográfico
    'Linea de producto': 'Product line',
    'Género': 'Gender',
    'Ciudad': 'City',
    'Tipo de pago': 'Payment',
    'Tipo de comprador': 'Customer type',
    # Si también calculaste COGS o Margen Bruto y quieres renombrar, añádelos aquí:
    # 'cogs': 'COGS_Original', # Si quieres mantener la original con otro nombre
    # 'Margen bruto': 'Gross margin percentage_Original', # Si quieres mantener la original con otro nombre
})

# Identificar las columnas originales que serán reemplazadas por las calculadas
# Estas son las columnas originales que tenían un formato numérico problemático
original_numeric_cols_to_replace = ['Total', 'Impuesto 5%', 'Ingreso bruto']

# Renombrar las columnas calculadas a los nombres finales deseados
# Esto reemplazará las columnas originales si existen, o creará las nuevas si no
df_limpio = df_limpio.rename(columns={
    'Subtotal_Calculado': 'Subtotal',
    'Impuesto_Calculado': 'Tax 5%', # Usar el nombre original deseado para el impuesto
    'Total_Calculado': 'Total',   # Usar el nombre original deseado para el total
    'Ingreso bruto_Calculado': 'Gross income', # Usar el nombre original deseado para el ingreso bruto
    # Si calculaste Margen Bruto:
    # 'Margen bruto_Calculado': 'Gross margin percentage', # Usar el nombre original deseado
})

# Ahora, eliminamos las columnas originales que fueron efectivamente reemplazadas
# y cualquier columna temporal que ya no necesitemos.
# No necesitamos listar 'Total', 'Impuesto 5%', 'Ingreso bruto' aquí si el renombrado
# anterior ya las sobrescribió. Solo listamos las que no queremos.
columnas_a_eliminar_final = [
    'Ingreso bruto_Original_Numeric', # Eliminar la versión numérica original temporal
    # Si decidiste NO mantener las originales de COGS/Margen Bruto renombradas con "_Original":
    # 'cogs',
    # 'Margen bruto',
]

# Eliminar las columnas si existen en el DataFrame
columnas_existentes_a_eliminar = [col for col in columnas_a_eliminar_final if col in df_limpio.columns]
df_limpio = df_limpio.drop(columns=columnas_existentes_a_eliminar)

# Volver a ejecutar el análisis para verificar las columnas finales
print("\nColumnas finales de df_limpio después de renombrar y eliminar:")
print(df_limpio.columns)
display(df_limpio.head())
display(df_limpio.info())

print("\nColumnas finales de df_limpio después de renombrar y eliminar:")
print(df_limpio.columns) # <--- VUELVE A VERIFICAR LA SALIDA AQUÍ después de ejecutar la Parte 5

In [ ]:
# Parte 6
import pandas as pd

# Asegurarse de que df_limpio tiene las columnas esperadas
required_cols = ['Fecha', 'Product line', 'Total']
for col in required_cols:
    if col not in df_limpio.columns:
        print(f"Error: La columna '{col}' no se encontró en el DataFrame df_limpio.")
        print("Columnas disponibles:", df_limpio.columns.tolist())
        # Puedes añadir un raise KeyError(f"Missing column: {col}") here if you want to stop execution
        # Si no quieres detener la ejecución, maneja este caso apropiadamente
        # Para este ejemplo, vamos a asumir que necesitamos detener la ejecución si faltan columnas clave.
        raise KeyError(f"Missing required column: '{col}'")


# Convertir la columna 'Fecha' a objetos datetime
# Asegúrate de que el nombre de la columna de fecha es EXACTAMENTE 'Fecha'
# Usa errors='coerce' para convertir valores no válidos a NaT (Not a Time)
df_limpio['Fecha'] = pd.to_datetime(df_limpio['Fecha'], errors='coerce')

# Opcional: Eliminar filas donde la conversión de fecha falló si es necesario para el análisis
# df_limpio.dropna(subset=['Fecha'], inplace=True)


# Agrupar datos y calcular ventas diarias por línea de producto
# Usando el nombre de columna renombrado 'Total'
ventas_diarias = df_limpio.groupby(['Fecha', 'Product line'])['Total'].sum().reset_index()

# Calcular ventas diarias promedio por línea de producto
ventas_diarias_promedio = ventas_diarias.groupby('Product line')['Total'].mean()

# Mostrar los resultados
print("\nVentas diarias promedio por línea de producto:")
display(ventas_diarias_promedio)

In [ ]:
# Parte 7
# Agrupar por línea de producto y sumar los ingresos totales
ingresos_por_linea = df_limpio.groupby('Product line')['Total'].sum()

# Encontrar la línea de producto con el mayor ingreso total
linea_mayor_ingreso = ingresos_por_linea.idxmax()

# Mostrar el resultado
print(f"La línea de producto con el mayor ingreso total es: {linea_mayor_ingreso}")

In [ ]:
# Parte 8
# Agrupar los datos por 'Product line' y 'Gender' y calcular la calificación promedio
calificacion_promedio_por_producto_genero = df_limpio.groupby(['Product line', 'Gender'])['Rating'].mean()

# Mostrar el resultado
display(calificacion_promedio_por_producto_genero)

In [ ]:
# Parte 9
# Filtrar para Yangon y Salud y belleza
yangon_belleza = df_limpio[(df_limpio['City'] == 'Yangon') & (df_limpio['Product line'] == 'Health and beauty')]

# Agrupar por género y sumar cantidad
ventas_belleza_por_genero = yangon_belleza.groupby('Gender')['Quantity'].sum()

# Mostrar el resultado
display(ventas_belleza_por_genero)

In [ ]:
# Parte 10
import matplotlib.pyplot as plt

# 1. Ventas diarias promedio por línea de producto
plt.figure(figsize=(10, 6))
ventas_diarias_promedio.plot(kind='bar', color='skyblue')
plt.title('Ventas diarias promedio por línea de producto')
plt.xlabel('Línea de producto')
plt.ylabel('Ventas diarias promedio')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('ventas_diarias_promedio.png')
plt.show()

# 2. Calificación promedio por línea de producto y género
plt.figure(figsize=(12, 6))
calificacion_promedio_por_producto_genero.unstack().plot(kind='bar', color=['lightcoral', 'lightblue'])
plt.title('Calificación promedio por línea de producto y género')
plt.xlabel('Línea de producto')
plt.ylabel('Calificación promedio')
plt.xticks(rotation=45, ha='right')
plt.legend(['Femenino', 'Masculino'])
plt.tight_layout()
plt.savefig('calificacion_promedio.png')
plt.show()

# 3. Ingresos totales por línea de producto
plt.figure(figsize=(10, 6))
ingresos_por_linea.plot(kind='bar', color='lightgreen')
plt.title('Ingresos totales por línea de producto')
plt.xlabel('Línea de producto')
plt.ylabel('Ingresos totales')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('ingresos_totales.png')
plt.show()

# 4. Productos de belleza vendidos en Yangon por género
plt.figure(figsize=(8, 6))
ventas_belleza_por_genero.plot(kind='bar', color=['pink', 'lightblue'])
plt.title('Productos de belleza vendidos en Yangon por género')
plt.xlabel('Género')
plt.ylabel('Cantidad vendida')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('productos_belleza_yangon.png')
plt.show()

In [ ]:
# Parte 11
import matplotlib.pyplot as plt

# 1. Ventas diarias promedio por línea de producto
plt.figure(figsize=(10, 6))
ventas_diarias_promedio.plot(kind='bar', color='skyblue')
plt.title('Ventas diarias promedio por línea de producto')
plt.xlabel('Línea de producto')
plt.ylabel('Ventas diarias promedio')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('ventas_diarias_promedio.png')
plt.show()

# 2. Calificación promedio por línea de producto y género
plt.figure(figsize=(12, 6))
calificacion_promedio_por_producto_genero.plot(kind='bar', color=['lightcoral', 'lightblue'])
plt.title('Calificación promedio por línea de producto y género')
plt.xlabel('Línea de producto')
plt.ylabel('Calificación promedio')
plt.xticks(rotation=45, ha='right')
plt.legend(calificacion_promedio_por_producto_genero.index.get_level_values('Gender').unique())
plt.tight_layout()
plt.savefig('calificacion_promedio.png')
plt.show()

# 3. Ingresos totales por línea de producto
plt.figure(figsize=(10, 6))
ingresos_por_linea.plot(kind='bar', color='lightgreen')
plt.title('Ingresos totales por línea de producto')
plt.xlabel('Línea de producto')
plt.ylabel('Ingresos totales')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('ingresos_totales.png')
plt.show()

# 4. Productos de belleza vendidos en Yangon por género
plt.figure(figsize=(8, 6))
ventas_belleza_por_genero.plot(kind='bar', color=['pink', 'lightblue'])
plt.title('Productos de belleza vendidos en Yangon por género')
plt.xlabel('Género')
plt.ylabel('Cantidad vendida')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('productos_belleza_yangon.png')
plt.show()